# Time Series Analysis

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

plt.style.use("seaborn-v0_8")
warnings.filterwarnings("ignore")

## a) Exploration

### i)

In [5]:
candidate_paths = [
    Path("files/time-series/air_traffic.csv"),
    Path("time-series/air_traffic.csv"),
    Path("../files/time-series/air_traffic.csv"),
]
for p in candidate_paths:
    if p.exists():
        data_path = p
        break
else:
    raise FileNotFoundError(f"air_traffic.csv not found in any of: {candidate_paths}")

air_raw = pd.read_csv(data_path)

air = air_raw.copy()
for col in air.columns:
    air[col] = pd.to_numeric(air[col].astype(str).str.replace(",", ""), errors="coerce")

air["Date"] = pd.to_datetime(dict(year=air["Year"], month=air["Month"], day=1))
air = air.sort_values("Date").reset_index(drop=True)

ts_flights = air.set_index("Date")["Flt"]
ts_passengers = air.set_index("Date")["Pax"]

print(f"Using data path: {data_path}")
print(f"Time range: {air['Date'].min().date()} to {air['Date'].max().date()}")
print(f"Months tracked: {len(air)}")
print(f"Total flights overall: {int(ts_flights.sum()):,}")

FileNotFoundError: [Errno 2] No such file or directory: 'files\\time-series\\air_traffic.csv'

### ii)

Strong yearly seasonality with summer peaks; flights collapse in spring 2020 and then recover but stay below the mid-2000s highs.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(ts_flights.index, ts_flights, color="tab:blue", label="Flights per month")
ax.set_title("Monthly US flights (domestic + international)")
ax.set_xlabel("Date")
ax.set_ylabel("Number of flights")
ax.legend()
plt.tight_layout()

### iii)

Flights and passengers move together and share the same seasonal pattern. Passenger counts trend upward more strongly (higher load factors) while flight counts stay flatter; both dip sharply in early 2020.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 4))
ax1.plot(ts_flights.index, ts_flights, label="Flights", color="tab:blue")
ax1.set_ylabel("Flights")
ax2 = ax1.twinx()
ax2.plot(ts_passengers.index, ts_passengers / 1e6, label="Passengers (millions)", color="tab:orange")
ax2.set_ylabel("Passengers (millions)")
ax1.set_title("Flights vs passengers over time")
ax1.set_xlabel("Date")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="upper left")
plt.tight_layout()

## b) Investigation
### i)

Yearly seasonal plots below highlight repeating summer peaks and winter lows. The pattern breaks in 2020 because of COVID-19, but later years return to the usual shape at slightly different levels.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=False)

sns.lineplot(data=air, x="Month", y="Flt", hue="Year", palette="turbo", legend=False, ax=axes[0])
axes[0].set_title("Flights seasonal plot by year")
axes[0].set_xlabel("Month")
axes[0].set_ylabel("Flights")

sns.lineplot(data=air, x="Month", y="Pax", hue="Year", palette="turbo", legend=False, ax=axes[1])
axes[1].set_title("Passengers seasonal plot by year")
axes[1].set_xlabel("Month")
axes[1].set_ylabel("Passengers")

for ax in axes:
    ax.set_xticks(range(1, 13))
    ax.grid(True, alpha=0.3)

plt.tight_layout()

### ii)

The correlation coefficient below quantifies how closely passengers and flight counts co-move (a value near 1 would indicate very similar dynamics).

In [ ]:
corr = ts_flights.corr(ts_passengers)
print(f"Correlation between passengers and flights: {corr:.3f}")

### iii)

Correlograms highlight the strongest recurring lags. We expect the largest positive spikes around a 12-month lag (and 24 months before differencing), showing yearly seasonality; after first and second differencing, short lags flatten while the seasonal peak remains most pronounced.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 10))
for idx, d in enumerate([0, 1, 2]):
    series = ts_flights.copy()
    for _ in range(d):
        series = series.diff().dropna()
    plot_acf(series, lags=24, ax=axes[idx])
    axes[idx].set_title(f"ACF of flights with {d} order differencing")
    axes[idx].set_xlabel("Lag (months)")
    axes[idx].set_ylabel("Correlation")

plt.tight_layout()

### iv)

STL decomposition (period 12) separates the trend, yearly seasonality, and residual component to check for stationarity of the remainder. The ADF test on the residuals (printed below) indicates whether the remaining series is stationary after removing trend and seasonality.

In [ ]:
from statsmodels.tsa.seasonal import STL

In [ ]:
stl = STL(ts_flights, period=12, robust=True)
stl_result = stl.fit()

fig = stl_result.plot()
fig.set_size_inches(10, 8)
plt.tight_layout()

adf_stat, adf_p, *_ = adfuller(stl_result.resid.dropna())
print(f"ADF statistic: {adf_stat:.3f}, p-value: {adf_p:.4f}")
if adf_p < 0.05:
    print("Residual appears stationary (reject null of unit root).")
else:
    print("Residual may not be fully stationary (fail to reject unit root).")

## c) Forecasting

### i)

Both flights and passengers series show clear yearly seasonality and a structural shock in 2020. Seasonal models (e.g., seasonal ARIMA) should forecast well when trained on stable periods, but forecasts must account for the pandemic outlier.

In [ ]:
from sktime.forecasting.arima import StatsModelsARIMA
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
train_path_candidates = [
    Path("files/time-series/flights_train.csv"),
    Path("time-series/flights_train.csv"),
    Path("../files/time-series/flights_train.csv"),
]
test_path_candidates = [
    Path("files/time-series/flights_test.csv"),
    Path("time-series/flights_test.csv"),
    Path("../files/time-series/flights_test.csv"),
]

for candidates, name in [(train_path_candidates, "flights_train.csv"), (test_path_candidates, "flights_test.csv")]:
    for p in candidates:
        if p.exists():
            if name == "flights_train.csv":
                train_path = p
            else:
                test_path = p
            break
    else:
        raise FileNotFoundError(f"{name} not found in: {candidates}")

print(f"Using train path: {train_path}")
print(f"Using test path: {test_path}")

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

for df in (train, test):
    if "Date" in df.columns:
        df["Date"] = pd.PeriodIndex(pd.to_datetime(df["Date"]), freq="M")
    else:
        df["Date"] = pd.PeriodIndex(pd.to_datetime(dict(year=df["Year"], month=df["Month"], day=1)), freq="M")
    df["Flt"] = pd.to_numeric(df["Flt"], errors="coerce")

y_train = pd.Series(train["Flt"].values, index=train["Date"])
y_test = pd.Series(test["Flt"].values, index=test["Date"])
fh = list(range(1, len(y_test) + 1))

def compute_metrics(y_true, y_pred):
    return {
        "RMSE": mean_squared_error(y_true, y_pred, square_root=True),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MAPE (%)": mean_absolute_percentage_error(y_true, y_pred) * 100,
    }

naive = NaiveForecaster(strategy="mean")
naive.fit(y_train)
naive_pred = naive.predict(fh=fh)
naive_metrics = compute_metrics(y_test, naive_pred)

candidate_orders = [
    (1, 0, 1),
    (1, 1, 1),
    (2, 1, 1),
    (2, 1, 2),
    (3, 1, 1),
    (3, 1, 2),
    (4, 1, 2),
    (5, 1, 1),
    (5, 1, 2),
    (12, 1, 1),
]

best_order, best_metrics, best_pred = None, None, None
for order in candidate_orders:
    try:
        model = StatsModelsARIMA(order=order, suppress_warnings=True, trend=None)
        model.fit(y_train)
        preds = model.predict(fh=fh)
        metrics = compute_metrics(y_test, preds)
    except Exception:
        continue
    if best_metrics is None or metrics["RMSE"] < best_metrics["RMSE"]:
        best_order, best_metrics, best_pred = order, metrics, preds

print("Naive (mean) forecaster:", naive_metrics)
if best_order is None:
    print("ARIMA search failed; please check library versions.")
    best_pred = naive_pred
else:
    print(f"Best ARIMA order: {best_order} with metrics {best_metrics}")
    if all(best_metrics[m] < naive_metrics[m] for m in ["RMSE", "MAE", "MAPE (%)"]):
        print("ARIMA beats the naive mean forecast on all metrics.")

if best_metrics is not None:
    metrics_df = pd.DataFrame(
        [naive_metrics, best_metrics], index=["Naive (mean)", f"ARIMA{best_order}"]
    )
else:
    metrics_df = pd.DataFrame([naive_metrics], index=["Naive (mean)"])

display(metrics_df)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(y_train.index.to_timestamp(), y_train, label="Train")
ax.plot(y_test.index.to_timestamp(), y_test, label="Test", color="black")
ax.plot(
    y_test.index.to_timestamp(),
    best_pred,
    label=f"Forecast ({'ARIMA ' + str(best_order) if best_order else 'naive'})",
    color="tab:green",
)
ax.set_title("Flights forecast")
ax.set_xlabel("Date")
ax.set_ylabel("Flights")
ax.legend()
plt.tight_layout()